# a couple of ways to do two-way ANOVA with Python
Here is my version of [this article](https://www.marsja.se/three-ways-to-carry-out-2-way-anova-with-python/)
There are a lot of errors, and so, I'd fixed it

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
datafile = "../../data/ToothGrowth.csv"
data = pd.read_csv(datafile)
data.sample(10)

In [ ]:
fig = interaction_plot(data.dose, data.supp, data.len,
             colors=['red','blue'], markers=['D','^'], ms=10)

# Calculation of Sum of Squares
The calculations of the sum of squares (the variance in the data) are quite simple using Python. 
First, we start with getting the sample size (N) and the degree of freedoms needed. 
We will use them later to calculate the mean square. 
After we have the degree of freedom we continue with the calculation of the sum of squares.

# Degrees of Freedom

In [ ]:
N = len(data.len)
df_a = len(data.supp.unique()) - 1
df_b = len(data.dose.unique()) - 1
df_axb = df_a*df_b 
df_w = N - (len(data.supp.unique())*len(data.dose.unique()))

# Sum of Squares
For the calculation of the sum of squares A, B and Total we will need to have the grand mean. Using Pandas DataFrame method mean on the dependent variable only will give us the grand mean:

In [ ]:
grand_mean = data['len'].mean()

# Sum of Squares A – supp
We start with calculation of Sum of Squares for the factor A (supp).

In [ ]:
ssq_a = sum([(data[data.supp ==l].len.mean()-grand_mean)**2 for l in data.supp])

# Sum of Squares B – dose
Calculation of the second Sum of Square, B (dose), is pretty much the same but over the levels of that factor.

In [ ]:
ssq_b = sum([(data[data.dose ==l].len.mean()-grand_mean)**2 for l in data.dose])

# Sum of Squares Total

In [ ]:
ssq_t = sum((data.len - grand_mean)**2)

# Sum of Squares Within (error/residual)
Next, we need to calculate the Sum of Squares Within which is sometimes referred to as error or residual.

In [ ]:
vc = data[data.supp == 'VC']
oj = data[data.supp == 'OJ']
vc_dose_means = [vc[vc.dose == d].len.mean() for d in vc.dose]
oj_dose_means = [oj[oj.dose == d].len.mean() for d in oj.dose]

In [ ]:
ssq_w = sum(((oj.len - oj_dose_means)**2) + sum((vc.len - vc_dose_means)**2))

# Sum of Squares interaction
Since we have a two-way design we need to calculate the Sum of Squares for the interaction of A and B.

In [ ]:
ssq_axb = ssq_t-ssq_a-ssq_b-ssq_w

# Mean Squares
We continue with the calculation of the mean square for each factor, the interaction of the factors, and within.

In [ ]:
# Mean Square A
ms_a = ssq_a/df_a

# Mean Square B
ms_b = ssq_b/df_b

# Mean Square AxB
ms_axb = ssq_axb/df_axb

# Mean Square Within/Error/Residual
ms_w = ssq_w/df_w

# F-ratio
The F-statistic is simply the mean square for each effect and the interaction 
divided by the mean square for within (error/residual).

In [ ]:
# F-ratio A
f_a = ms_a/ms_w
# F-ratio B
f_b = ms_b/ms_w
# F-ratio AxB
f_axb = ms_axb/ms_w

# Obtaining p-values
We can use the scipy.stats method f.sf to check if our obtained F-ratios is above the critical value. Doing that we need to use our F-value for each effect and interaction as well as the degrees of freedom for them, and the degree of freedom within.

In [ ]:
p_a = stats.f.sf(f_a, df_a, df_w)
p_b = stats.f.sf(f_b, df_b, df_w)
p_axb = stats.f.sf(f_axb, df_axb, df_w)

The results are, right now, stored in a lot of variables. To obtain a more readable result we can create a DataFrame that will contain our ANOVA table.

In [ ]:
results = {'sum_sq':[ssq_a, ssq_b, ssq_axb, ssq_w],
           'df':[df_a, df_b, df_axb, df_w],
           'F':[f_a, f_b, f_axb, 'NaN'],
            'PR(&gt;F)':[p_a, p_b, p_axb, 'NaN']}
columns=['sum_sq', 'df', 'F', 'PR(&gt;F)']
aov_table1 = pd.DataFrame(results, columns=columns,
                          index=['supp', 'dose', 
                          'supp:dose', 'Residual'])

So, common software, such as SPSS has eta squared as output. However, eta squared is an overestimation of the effect. To get a less biased effect size measure we can use omega squared. The following two functions add eta squared and omega squared to the above DataFrame that contains the ANOVA table.

In [ ]:
def eta_squared(aov):
    aov['eta_sq'] = 'NaN'
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    return aov
def omega_squared(aov):
    mse = aov['sum_sq'][-1]/aov['df'][-1]
    aov['omega_sq'] = 'NaN'
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*mse))/(sum(aov['sum_sq'])+mse)
    return aov
eta_squared(aov_table1)
omega_squared(aov_table1)
aov_table1




# Two-way ANOVA using Statsmodels
There is, of course, a much easier way to do Two-way ANOVA with Python. We can use Statsmodels which have a similar model notation as many R-packages (e.g., lm). We start with the formulation of the model:

In [ ]:
formula = 'len ~ C(supp) + C(dose) + C(supp):C(dose)'
model = ols(formula, data).fit()
aov_table = anova_lm(model, typ=2)

Statsmodels does not calculate effect sizes for us. My functions above can, again, be used and will add omega and eta squared effect sizes to the ANOVA table. Actually, I created these two functions to enable calculation of omega and eta squared effect sizes on the output of Statsmodels anova_lm method. Note, statsmodels can be installed with e.g. pip or conda.

In [ ]:
eta_squared(aov_table)
omega_squared(aov_table)
aov_table.round(3)

What is neat with using statsmodels is that we can also do some diagnostics. It is, for instance, very easy to take our model fit (the linear model fitted with the OLS method) and get a Quantile-Quantile (QQplot):

In [ ]:
res = model.resid 
fig = sm.qqplot(res, stats.t, fit=True, line="45")
plt.show()

# Two-Way ANOVA using Pingouin
Here, Pingouin offers a very easy way to for ANOVA in Python.
Unfortunatelly, it doesn't work on the Kaggle site (in reason of this library, which i use), but you can copy these few lines of code...

In [ ]:
import pingouin as pg
data = pd.read_csv(datafile)
data.head()

In [ ]:
aov = pg.anova(data, dv='len', between=['supp', 'dose'], detailed=True)
aov

In [ ]:
# Thank you!